# First appraoches

In [14]:
from nba_api.stats.endpoints import leaguedashplayerclutch
import inspect

# Print the parameters accepted by LeagueDashPlayerClutch
print(inspect.signature(leaguedashplayerclutch.LeagueDashPlayerClutch.__init__))

(self, ahead_behind='Ahead or Behind', clutch_time='Last 5 Minutes', last_n_games='0', measure_type_detailed_defense='Base', month='0', opponent_team_id=0, pace_adjust='N', per_mode_detailed='Totals', period='0', plus_minus='N', point_diff='5', rank='N', season='2024-25', season_type_all_star='Regular Season', college_nullable='', conference_nullable='', country_nullable='', date_from_nullable='', date_to_nullable='', division_simple_nullable='', draft_pick_nullable='', draft_year_nullable='', game_scope_simple_nullable='', game_segment_nullable='', height_nullable='', league_id_nullable='', location_nullable='', outcome_nullable='', po_round_nullable='', player_experience_nullable='', player_position_abbreviation_nullable='', season_segment_nullable='', shot_clock_range_nullable='', starter_bench_nullable='', team_id_nullable='', vs_conference_nullable='', vs_division_nullable='', weight_nullable='', proxy=None, headers=None, timeout=30, get_request=True)


In [15]:
from nba_api.stats.endpoints import leaguedashplayerclutch
import pandas as pd

# Get clutch stats with the correct parameters
clutch_stats = leaguedashplayerclutch.LeagueDashPlayerClutch(
    ahead_behind='Ahead or Behind',
    clutch_time='Last 5 Minutes',
    point_diff='5',
    season='2023-24',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame'
)

# Convert to DataFrame
clutch_df = clutch_stats.get_data_frames()[0]

In [16]:
clutch_df.shape

(469, 67)

# Get all clutch plays and select top performers

In [26]:
from nba_api.stats.endpoints import leaguedashplayerclutch
import pandas as pd
import time
import random

def get_clutch_stats_for_season(season='2023-24', min_minutes=0):
    """
    Get clutch stats for all players in a given season with no filtering
    
    Parameters:
    season (str): Season in format '2023-24'
    min_minutes (float): Minimum clutch minutes to include (0 = no filtering)
    
    Returns:
    pandas.DataFrame: DataFrame with clutch stats for all players
    """
    try:
        print(f"Fetching clutch stats for season {season}...")
        
        # Parameters for clutch time (last 5 min, game within 5 points)
        clutch_stats = leaguedashplayerclutch.LeagueDashPlayerClutch(
            season=season,
            season_type_all_star='Regular Season',
            measure_type_detailed_defense='Advanced',
            clutch_time='Last 5 Minutes',
            point_diff='5',
            per_mode_detailed='Totals'  # Get total stats instead of per game
        )
        
        # Get the data
        df = clutch_stats.get_data_frames()[0]
        
        # Force the SEASON column
        df['SEASON'] = season
        
        # Filter for minimum minutes if specified
        if min_minutes > 0:
            filtered_df = df[df['MIN'] >= min_minutes].copy()
            print(f"Found {len(filtered_df)} players with ≥{min_minutes} clutch minutes in {season} (filtered from {len(df)} total)")
        else:
            filtered_df = df.copy()
            print(f"Found {len(filtered_df)} players with clutch data in {season}")
        
        # Calculate clutch impact score
        filtered_df['CLUTCH_IMPACT'] = (
            filtered_df['NET_RATING'] * 0.5 +
            filtered_df['USG_PCT'] * 0.2 +
            filtered_df['TS_PCT'] * 100 * 0.2 +
            filtered_df['PIE'] * 100 * 0.1
        )
        
        return filtered_df
        
    except Exception as e:
        print(f"Error getting clutch stats for season {season}: {e}")
        import traceback
        traceback.print_exc()
        return None

def process_multiple_seasons(start_year, end_year):
    """
    Process multiple seasons of clutch data with no intermediate saving
    
    Parameters:
    start_year (int): Starting year (e.g., 2018 for 2018-19 season)
    end_year (int): Ending year (e.g., 2023 for 2023-24 season)
    
    Returns:
    pandas.DataFrame: Combined DataFrame with clutch stats for all seasons
    """
    all_seasons_data = []
    
    for year in range(start_year, end_year + 1):
        # Create season string (e.g., '2023-24')
        season = f"{year}-{str(year+1)[-2:]}"
        
        print(f"\n=== Processing {season} season ===")
        
        # Get clutch stats for this season - no minimum minutes filter
        season_data = get_clutch_stats_for_season(season, min_minutes=0)
        
        if season_data is not None and not season_data.empty:
            # Add to the combined data
            all_seasons_data.append(season_data)
            
            print(f"Added {len(season_data)} players from {season} to dataset")
            
            # Add delay between seasons to avoid rate limiting
            if year < end_year:
                delay = random.uniform(2, 4)
                print(f"Waiting {delay:.2f} seconds before next season...")
                time.sleep(delay)
        else:
            print(f"No data collected for {season}")
    
    # Combine all seasons
    if all_seasons_data:
        combined_data = pd.concat(all_seasons_data, ignore_index=True)
        print(f"\nTotal: {len(combined_data)} player-season records across {len(all_seasons_data)} seasons")
        return combined_data
    else:
        print(f"No data collected for any seasons")
        return pd.DataFrame()

def get_top_clutch_performers(df, n=50, min_games=10):
    """
    Get top clutch performers from a DataFrame with more robust filtering
    
    Parameters:
    df (pandas.DataFrame): DataFrame with clutch stats
    n (int): Number of top performers to return
    min_games (int): Minimum games played in clutch situations
    
    Returns:
    pandas.DataFrame: DataFrame with top clutch performers
    """
    # Filter for minimum games played
    filtered_df = df[df['GP'] >= min_games].copy()
    
    print(f"Filtering for players with at least {min_games} clutch games")
    print(f"Before: {len(df)} players, After: {len(filtered_df)} players")
    
    # Make sure we have clutch impact score
    if 'CLUTCH_IMPACT' not in filtered_df.columns:
        filtered_df['CLUTCH_IMPACT'] = (
            filtered_df['NET_RATING'] * 0.5 +
            filtered_df['USG_PCT'] * 0.2 +
            filtered_df['TS_PCT'] * 100 * 0.2 +
            filtered_df['PIE'] * 100 * 0.1
        )
    
    # Sort by clutch impact score and get top n
    top_players = filtered_df.sort_values('CLUTCH_IMPACT', ascending=False).head(n)
    
    # Select relevant columns for display
    display_cols = [
        'PLAYER_NAME', 'TEAM_ABBREVIATION', 'SEASON', 
        'GP', 'MIN', 'NET_RATING', 'OFF_RATING', 'DEF_RATING',
        'USG_PCT', 'TS_PCT', 'PIE', 'CLUTCH_IMPACT'
    ]
    
    # Get columns that exist in the DataFrame
    available_cols = [col for col in display_cols if col in top_players.columns]
    
    return top_players[available_cols]


In [27]:
# Example usage
# Get data for multiple seasons (with no intermediate saving)
all_data = process_multiple_seasons(2018, 2023)

if not all_data.empty:
    # Get top clutch performers across all seasons (min 10 games)
    top_performers = get_top_clutch_performers(all_data, n=50, min_games=10)
    
    print("\nTop 50 Clutch Performers (2018-2023) with at least 10 clutch games:")
    print(top_performers)
    
    # Save only the final results
    final_filename = "top_clutch_performers_2018-2023.csv"
    top_performers.to_csv(final_filename, index=False)
    print(f"Saved top performers to {final_filename}")
    
    # Also save the complete dataset
    complete_filename = "all_clutch_data_2018-2023.csv"
    all_data.to_csv(complete_filename, index=False)
    print(f"Saved complete dataset to {complete_filename}")


=== Processing 2018-19 season ===
Fetching clutch stats for season 2018-19...
Found 451 players with clutch data in 2018-19
Added 451 players from 2018-19 to dataset
Waiting 2.95 seconds before next season...

=== Processing 2019-20 season ===
Fetching clutch stats for season 2019-20...
Found 456 players with clutch data in 2019-20
Added 456 players from 2019-20 to dataset
Waiting 2.16 seconds before next season...

=== Processing 2020-21 season ===
Fetching clutch stats for season 2020-21...
Found 454 players with clutch data in 2020-21
Added 454 players from 2020-21 to dataset
Waiting 3.65 seconds before next season...

=== Processing 2021-22 season ===
Fetching clutch stats for season 2021-22...
Found 496 players with clutch data in 2021-22
Added 496 players from 2021-22 to dataset
Waiting 3.26 seconds before next season...

=== Processing 2022-23 season ===
Fetching clutch stats for season 2022-23...
Found 451 players with clutch data in 2022-23
Added 451 players from 2022-23 to d

In [41]:
all_data[all_data['PLAYER_NAME'] == 'Aaron Gordon'].groupby(['PLAYER_NAME', 'SEASON']).agg({
    'NET_RATING': 'mean',
    'GP': 'sum',
    'MIN': 'sum'
}).reset_index()

,PLAYER_NAME,SEASON,NET_RATING,GP,MIN
0,Aaron Gordon,2018-19,-0.4,43,2.9
1,Aaron Gordon,2019-20,-6.6,27,3.3
2,Aaron Gordon,2020-21,7.7,19,2.9
3,Aaron Gordon,2021-22,11.0,32,4.0
4,Aaron Gordon,2022-23,16.0,28,3.7
5,Aaron Gordon,2023-24,19.4,38,3.2


# Select Top 10 Clutch Players Based on +/-

In [42]:
# Example usage
# Get data for multiple seasons
all_data = process_multiple_seasons(2000, 2025)

if not all_data.empty:
    # Get top clutch performers across all seasons (min 10 games)
    top_performers = get_top_clutch_performers(all_data, n=10, min_games=10)
    
    print(f"\nTop 10 Clutch Performers (2000-2023) with at least 10 clutch games:")
    print(top_performers)
    
    # Save only the final results
    # final_filename = "top_clutch_performers_2018-2023.csv"
    # top_performers.to_csv(final_filename, index=False)
    # print(f"Saved top performers to {final_filename}")
    
    # # Also save the complete dataset
    # complete_filename = "all_clutch_data_2018-2023.csv"
    # all_data.to_csv(complete_filename, index=False)
    # print(f"Saved complete dataset to {complete_filename}")


=== Processing 2000-01 season ===
Fetching clutch stats for season 2000-01...
Found 390 players with clutch data in 2000-01
Added 390 players from 2000-01 to dataset
Waiting 3.97 seconds before next season...

=== Processing 2001-02 season ===
Fetching clutch stats for season 2001-02...
Found 398 players with clutch data in 2001-02
Added 398 players from 2001-02 to dataset
Waiting 2.69 seconds before next season...

=== Processing 2002-03 season ===
Fetching clutch stats for season 2002-03...
Found 386 players with clutch data in 2002-03
Added 386 players from 2002-03 to dataset
Waiting 2.17 seconds before next season...

=== Processing 2003-04 season ===
Fetching clutch stats for season 2003-04...
Found 388 players with clutch data in 2003-04
Added 388 players from 2003-04 to dataset
Waiting 2.20 seconds before next season...

=== Processing 2004-05 season ===
Fetching clutch stats for season 2004-05...
Found 417 players with clutch data in 2004-05
Added 417 players from 2004-05 to d

In [43]:
top_performers

,PLAYER_NAME,TEAM_ABBREVIATION,SEASON,GP,MIN,NET_RATING,OFF_RATING,DEF_RATING,USG_PCT,TS_PCT,PIE,CLUTCH_IMPACT
7309,Luc Mbah a Moute,HOU,2017-18,13,1.3,67.9,142.9,75.0,0.098,1.078,0.150,57.0296
6458,Mike Dunleavy,CHI,2015-16,11,1.5,75.9,137.2,61.3,0.143,0.680,0.106,52.6386
9806,Andre Drummond,CHI,2023-24,14,2.1,60.8,140.0,79.2,0.125,0.788,0.216,48.3450
5010,Greg Smith,HOU,2012-13,10,2.8,45.4,118.8,73.3,0.097,0.962,0.261,44.5694
7091,Chris Paul,HOU,2017-18,21,2.6,50.1,145.5,95.5,0.324,0.797,0.302,44.0748
10587,Nicolas Batum,LAC,2024-25,11,3.0,25.8,104.2,78.4,0.013,1.500,0.085,43.7526
9116,Kelly Olynyk,DET,2021-22,11,0.5,56.7,140.0,83.3,0.250,0.798,-0.080,43.5600
1638,Christian Laettner,MIA,2004-05,15,2.4,53.5,121.5,68.1,0.170,0.689,0.262,43.1840
8911,Chris Paul,PHX,2021-22,29,3.1,52.0,138.4,86.4,0.303,0.689,0.300,42.8406
9575,Juan Toscano-Anderson,UTA,2022-23,11,1.5,42.5,150.0,107.5,0.068,1.042,0.067,42.7736


# Analyze Players to pin point attributes

In [ ]:
print(f'Players: {player or player in top_performers.PLAYER_NAME.valuesf}')

SyntaxError: f-string: expecting '=', or '!', or ':', or '}' (4018431171.py, line 1)